# Step 5: Spots Only Model
In this step we will take the transmission spectrum and analyze the signature with a spot contamination model and no atmosphere.

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
from chromatic import *
import emcee
from scipy.stats import chi2
from scipy.stats import norm
import corner

First we need to read in (or simulate) a WFC3 transmission spectrum

In [5]:
data_wave = np.array([1.126,1.144,1.163,1.181,
                          1.200,1.218,1.237,1.255,1.274,1.292,
                          1.311,1.329,1.348,1.366,1.385,
                          1.403,1.422,1.440,1.459,1.477,1.496,
                          1.514,1.533,1.551,1.57,1.588,
                          1.607,1.625])
data_flux = np.array([2808,2815,2745,2720,2777,2671,
                      2731,2691,2720,2853,2691,2806,
                      2976,2952,2873,2780,2836,2971,
                      2741,2872,2752,2794,2788,2754,
                      2718,2828,2759,2745])/1e6
data_err = 46.05e-6

Now we need to define a spot contamination model, the log probability function, and a wrapper for emcee

In [ ]:
def spot_transmission_model(params,w,err = 1e-5,
                            plot=False,N_transits=1,
                            **kwargs):
    
    rp_rstar, fspot, Tspot, Tamb = params
    transit_depth = rp_rstar**2
    err = err/np.sqrt(N_transits)
        
    s_spot = get_phoenix_photons(temperature=float(Tspot), wavelength=w, logg=4.4, metallicity=0.0)
    s_amb = get_phoenix_photons(temperature=float(Tamb), wavelength=w, logg=4.4, metallicity=0.0)
    
    flux_ratio = s_spot[1]/s_amb[1]
    top = 1. # technically this is only if f_tra = 0
    bottom = (1. - fspot) + fspot * flux_ratio
    d_spot = ((top / bottom) - 1.) * transit_depth
    
    # model_spec = (transit_depth + d_spot)*1e6
    model_spec = d_spot*1e6
    noisy_spec = np.random.normal(model_spec,err*1e6) 
    
    if plot:
        plt.figure(figsize=(5,3))
        plt.plot(w,model_spec,color='k',label='Input model')
        plt.errorbar(w,noisy_spec,yerr = err,fmt='o',color='blue',alpha=0.75,label='Simulated observation')
        plt.legend()
    
    return noisy_spec, model_spec

In [ ]:
def lnprob(parameters=None,**kwargs):

    rp_rstar,f_spot,T_spot,T_amb = parameters

    if (0.0<=f_spot<=0.5) and (2300.0<=T_spot<=8000.0) and (2300.0<=T_amb<=8000.0):

        ln_like=0.0
        
        "Teff likelihood"
        Teff_model = (f_spot*(T_spot**4.) + (1.-f_spot)*(T_amb**4.))**(1./4.)    
        chisq_Teff = (Teff_data - Teff_model)**2./(T_err)**2.
        err_weight_Teff = 1./np.sqrt(2.*np.pi*(T_err))
        ln_like += (err_weight_Teff - 0.5*chisq_Teff)
        
        "Transmission likelihood"
        _, model = spot_transmission_model(params = parameters, w = data_wave,err = data_err)
        chisq = np.nansum((data_flux-model)**2/(data_err**2))
        err_weight = np.nansum(1./np.sqrt(2.*np.pi*(data_err)))
        ln_like += (err_weight - 0.5*chisq)
        
    else:
        ln_like = -np.inf

    return ln_like

In [ ]:
def do_mcmc(label='oops you didnt label your samples :sadface:',
            nsteps=100,burnin=25,ndim=4,nwalkers=100,**kwargs):
    
    # these are initial parameters
    rp_rstar_init = np.random.uniform(rprstar-0.0005, rprstar+0.0005, nwalkers)
    fspot_init = np.random.uniform(0.01, 0.4, nwalkers)
    Tspot_init = np.random.uniform(2301., Teff_data-400, nwalkers)
    Tamb_init = np.random.uniform(Teff_data, Teff_data+100, nwalkers)
    p0 = np.transpose([rp_rstar_init, fspot_init, Tspot_init, Tamb_init])

    # set up file saving for the samples when finished
    filename = f"{label}.h5"
    backend = emcee.backends.HDFBackend(filename)
    backend.reset(nwalkers, ndim)
    # Initialize and run the sampler
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, backend=backend)
    result = sampler.run_mcmc(p0, nsteps,store=True)
    samples = sampler.chain[:, burnin:, :].reshape((-1, ndim)).T

    for i in range(len(samples)):
        tau_f = emcee.autocorr.integrated_time(samples[i])
        print('(Nsteps-burnin)*nwalkers/tau=',(nsteps-burnin)*nwalkers/tau_f)
    
    return samples

In [ ]:
data_wave = np.linspace(0.4,1.0,50) * u.micron

nsteps = 3000
ntransits = 1

planet_name = 'BD'
Teff_data = 3600.0
T_err = 50.0
rprstar = 0.05
data_err = 5e-5

fspot = 0.05
Tspot = 3000.0
Tamb = 3900.0
f1_params = [rprstar,0.05,Tspot,Tamb]
f2_params = [rprstar,0.25,Tspot,Tamb]

data_flux,data_model1 = spot_transmission_model(params = f1_params,
                                    w = data_wave,N_transits=ntransits,
                                    err=data_err,plot=False)
data_flux,data_model2 = spot_transmission_model(params = f2_params,
                                    w = data_wave,N_transits=ntransits,
                                    err=data_err,plot=False)

# chisq_model = np.nansum((data_flux-data_model)**2/((data_err/np.sqrt(ntransits))**2))
# degrees_of_freedom = len(data_flux)-4
# pval_model = chi2.sf(chisq_model, degrees_of_freedom)
# nsigma_model = norm.isf(pval_model)

# flat_line = np.median(data_flux)
# chisq_line = np.nansum((data_flux-flat_line)**2/((data_err/np.sqrt(ntransits))**2))
# pval_line = chi2.sf(chisq_line, degrees_of_freedom)
# nsigma_line = norm.isf(pval_line)

plt.figure(figsize=(9,2.5))
# plt.title('Transit Spectrum with '+f'fspot={fspot}, Tspot={int(Tspot)}K')
# plt.errorbar(data_wave,data_flux,data_err*1e6,fmt='o',alpha=0.8,label='Simulated Data',ms=3)
# plt.plot(data_wave,data_model1,c='blue',alpha=0.6,zorder=-100,label=f'5% Spot Coverage')
plt.plot(data_wave,data_model2,c='black',alpha=0.6,zorder=-100,label=f'25% Spot Coverage')
# plt.plot(data_wave,[flat_line]*len(data_wave),c='k',label=f'Flat line ruled out at {nsigma_line:.1f}-sigma')
plt.xlabel(r'Wavelength ($\mu$m)')
# plt.ylim(0,None)
plt.ylabel('ppm')
plt.title('Spot Contamination (25% Coverage)')
# plt.legend(loc='upper right')
plt.savefig('../figs/planet_model.png',dpi=400)
plt.show()

In [ ]:
samples = do_mcmc(label = f'TOI2119b_optimistic_TLSE_mcmc',
                  nsteps = nsteps, burnin = int(0.25*nsteps))

In [ ]:
rp_sam, fspot_sam, Tspot_sam, Tphot_sam = samples
for sample in samples:
    percentiles = np.percentile(sample,[16.,50.,84.])
    print(percentiles)

hires_wave = np.linspace(np.min(data_wave),np.max(data_wave),300)

fig,axs = plt.subplots(2,1,figsize=(6,4),sharex=True,gridspec_kw=dict(height_ratios=[1,0.3]))
axs[0].set_title(f'Simulated Transit Depth Spectrum of {planet_name}')
axs[0].errorbar(data_wave,data_flux,yerr=data_err,fmt='o',zorder=100,color='royalblue',alpha=0.75,label='Simulated Data')
# axs[0].plot(hires_wave,hires_bestmodel,zorder=10,color='darkred',linewidth=0.5)
axs[0].set_ylabel('Transit Depth')
# axs[0].set_ylim(0.0025,0.0032)
# axs[0].legend(fontsize=10)
# axs[1].set_title('1000 samples (0.5% of the models)')
axs[1].set_ylabel('Signal-to-noise')
axs[1].set_xlabel('Wavelength (micron)')
# axs[1].set_ylim(0,20)
axs[1].scatter(data_wave,np.abs((data_flux-np.median(data_flux))/data_err),color='k',s=3)

for k in range(0,1000):
        
    j = np.random.randint(low=0,high=(len(Tspot_sam)-1))
    these_parameters = [rp_sam[j],fspot_sam[j],Tspot_sam[j],Tphot_sam[j]]
    _,hires_model = spot_transmission_model(params = these_parameters, w = hires_wave)
    
    if k == 0:
        axs[0].plot(hires_wave,hires_model,c='pink',alpha=0.01,zorder=1,label='Spot Characteristics Models') # this will be the input wavelength from the order in question
    
    axs[0].plot(hires_wave,hires_model,c='pink',alpha=0.01,zorder=1) # this will be the input wavelength from the order in question

axs[0].legend()
plt.savefig(f'TOI2119b_optimistic_TLSE_mcmc.png')

In [ ]:
variable_names = [r'R$_p$/R$_*$',r'$f_{\rm spot}$',r'T$_{\rm spot}$',r'T$_{\rm amb}$']
#make the corner plot
figure = corner.corner(samples.T, labels=variable_names,
                       quantiles=[0.16, 0.5, 0.84],
                       show_titles=True, title_kwargs={"fontsize": 20})

# Extract the axes, loop through to set all the limits
axes = np.array(figure.axes).reshape((4, 4))
for yi in range(4):
    ax = axes[yi, 0]
    # ax.set_xlim(0.220,0.225)
    
    ax.axvline(True_params[0],color='red')
    ax = axes[1, 1]
    ax.set_xlim(0.0,0.30)
    ax.axvline(True_params[1],color='red')
    ax = axes[2, 1]
    ax.set_xlim(0.0,0.30)
    ax.axvline(True_params[1],color='red')
    ax = axes[3, 1]
    ax.axvline(True_params[1],color='red')
    ax.set_xlim(0.0,0.30)
    
    ax = axes[2, 2]
    ax.set_xlim(2500,3600)
    ax.axvline(True_params[2],color='red')
    ax = axes[3, 2]
    ax.set_xlim(2500,3600)
    ax.axvline(True_params[2],color='red')
    
    ax = axes[3, 3]
    ax.set_xlim(3200,4000)
    ax.axvline(True_params[3],color='red')
    
    
plt.savefig('optimistic_corner.png')